In [1]:
import json
import outlines
from transformers import AutoTokenizer
import torch
from textwrap import dedent

In [2]:
model_name = "microsoft/Phi-3-medium-4k-instruct"
model = outlines.models.transformers(
    model_name,
    model_kwargs={
        'trust_remote_code': True
    })
tokenizer = AutoTokenizer.from_pretrained(model_name)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
with open("../examples.json",'r') as fin:
    complaint_data = json.loads(fin.read())

In [4]:
from pydantic import BaseModel, Field, constr
from enum import Enum


class Department(str, Enum):
    clothing = "clothing"
    electronics = "electronics"
    kitchen = "kitchen"
    automotive = "automotive"

class ComplaintData(BaseModel):
    first_name: str
    last_name: str
    order_number: str = Field(pattern=r'[ADZ][0-9]{2}-[0-9]{4}')
    department: Department
    
complaint_processor = outlines.generate.json(model, ComplaintData)

In [5]:
def create_prompt(complaint):
    complaint_messages = [
        {
        'role': 'user',
        'content': f"""
        You are a complaint processing assistent, you aim is to process complaints and return the following intformation in this JSON format:
        {{
            'first_name': <first name>,
            'last_name': <last name>,
            'order number': <order number has the following format (ADZ)XX-XXXXX>,
            'department': <{"|".join([e.value for e in Department])}>,
        }}
        """},
        {'role': 'assistant',
         'content': "I undersand and will process the complaints in the JSON format you described"
        },
        {'role': 'user',
        'content': complaint['message']
        }
    ]
    complaint_prompt = tokenizer.apply_chat_template(complaint_messages, tokenize=False)
    return complaint_prompt

In [7]:
results = []
for complaint in complaint_data[0:10]:
    prompt = create_prompt(complaint)
    result = complaint_processor(prompt)
    results.append(result)

You are not running the flash-attention implementation, expect numerical differences.


In [12]:
idx = 4
complaint_data[idx]['message']

'Hi, my name is Sarah Collins.I recently ordered your SmartWidget, but it hasn stopped working entirely! I just purchased the RapidCharge battery pack that does not charge at all.My order was Z123456'

In [14]:
results[idx].json()

'{"first_name":"Sarah","last_name":"Collins","order_number":"Z12-3456","department":"electronics"}'